In [1]:
import scipy as sp
import numpy as np
import copy
import numpy.random as rnd

The gradient (and loss) as a function :

In [85]:
def GradERM(X, y, w, v, sigmafunc, Dsigmafunc, LambdaRegularization):
    zw = np.matmul(X, w)
    zv = np.matmul(X, v)
    GradwTotal = np.matmul(np.transpose(X), - np.divide(y - zw,2*sigmafunc(zv))) + X.shape[0]*LambdaRegularization*w
    #print(np.matmul(np.transpose(X), np.divide(y - zw,2*sigmafunc(zv))))
    #print(X.shape[0]*LambdaRegularization*w)
    GradvTotal = (np.matmul(np.transpose(X), np.multiply((np.divide(1,(2*sigmafunc(zv))) - np.divide(np.power((y - zw),2),(2*np.power(sigmafunc(zv),2)))), Dsigmafunc(zv))) 
                  + X.shape[0]*LambdaRegularization*v)
    print(np.matmul(np.transpose(X), np.multiply((np.divide(1,(2*sigmafunc(zv))) - np.divide(np.power((y - zw),2),(2*np.power(sigmafunc(zv),2)))), Dsigmafunc(zv))))
    print(X.shape[0]*LambdaRegularization*v)
    return(np.stack((GradwTotal, GradvTotal), axis = 1))

def LossERM(X, y, w, v, sigmafunc, LambdaRegularization):
    zw = np.matmul(X, w)
    zv = np.matmul(X, v)
    return( np.sum(np.divide(np.power((y - zw),2),(2*sigmafunc(zv))) + np.log(sigmafunc(zv))/2, 0) + X.shape[0]*LambdaRegularization*( np.dot(w,w) + np.dot(v,v) )/2 )

The GD step

In [64]:
def GDStepERM(X, y, wv, sigmafunc, Dsigmafunc, LambdaRegularization, LearningRate):
    wv = wv - LearningRate*GradERM(X, y, wv[:,0], wv[:,1], sigmafunc, Dsigmafunc, LambdaRegularization)
    return(LossERM(X, y, wv[:,0], wv[:,1], sigmafunc, LambdaRegularization))

Full GD function

In [56]:
def GDERM(X, y, wv, sigmafunc, Dsigmafunc, LambdaRegularization = 1, LearningRate = 0.02, MaxIter = 1e4, EpsConvergence = 1e-6, Verbose = True, VerboseRate = 100):
    Conv = 1
    NIter = 0
    Losses = [LossERM(X, y, wv[:,0], wv[:,1], sigmafunc, LambdaRegularization)]
    print("Iteration %s" % NIter)
    print("Current loss %s" % Losses[NIter])
    while(NIter < MaxIter and Conv > EpsConvergence):
        Losses.append(GDStepERM(X, y, wv, sigmafunc, Dsigmafunc, LambdaRegularization, LearningRate))
        NIter = NIter + 1
        if(Verbose and NIter%VerboseRate == 0):
            print("Iteration %s" % NIter)
            print("Current loss %s" % Losses[NIter])
    return(np.array(Losses))
        

Sigma Functions for $\sigma(z_v) = z_v^2$

In [57]:
def sigmaSquare(zv):
    return(np.power(zv,2))

def DsigmaSquare(zv):
    return(2*zv)

Main variables 

In [86]:
d = 100
M = 1000
alpha = M/d
LearningRate = 0.002
LambdaRegularization = 1
X = rnd.normal(0, 1/np.sqrt(d), size = (M, d))
wvTrue = rnd.normal(0, 1, size = (d, 2))
wvLearned = rnd.normal(0, 1, size = (d, 2))
yTrue = np.matmul(X, wvTrue[:,0])
yNoisy = yTrue + rnd.normal(0, np.sqrt(sigmaSquare(np.matmul(X, wvTrue[:,1]))))

Runner

In [84]:
Losses = GDERM(X, yNoisy, wvLearned, sigmaSquare, DsigmaSquare, MaxIter = 10, VerboseRate = 1)
wvLearned - wvTrue

Iteration 0
Current loss 4534958.234508912
Iteration 1
Current loss 6.013040452916795e+18
Iteration 2
Current loss 6.013040452916795e+18
Iteration 3
Current loss 6.013040452916795e+18
Iteration 4
Current loss 6.013040452916795e+18
Iteration 5
Current loss 6.013040452916795e+18
Iteration 6
Current loss 6.013040452916795e+18
Iteration 7
Current loss 6.013040452916795e+18
Iteration 8
Current loss 6.013040452916795e+18
Iteration 9
Current loss 6.013040452916795e+18
Iteration 10
Current loss 6.013040452916795e+18


array([[ 0.97431054, -0.55971636],
       [ 0.82990991,  1.64838314],
       [ 0.66572968,  0.29514521],
       [ 1.95617735, -0.32785316],
       [-1.57647027,  0.52134877],
       [-0.09403461, -1.23807703],
       [ 0.53458549, -0.91356865],
       [-0.98080984,  1.44111593],
       [ 0.41894114, -1.16544456],
       [-1.02120811,  0.93186461],
       [ 0.30950369,  0.22425465],
       [-0.10892243, -0.80888278],
       [ 1.03176381, -0.62422609],
       [-0.06695014, -0.87876011],
       [ 0.95718649, -0.15565045],
       [-1.02127816,  1.21388188],
       [ 0.14439771, -2.81877275],
       [-0.65513699, -0.00380717],
       [ 0.69088903,  1.37042919],
       [-1.37150003,  0.31312519],
       [ 0.37317116, -0.56668835],
       [-0.25368838, -0.74148329],
       [ 2.05944799, -1.77210348],
       [-0.2683296 ,  1.04972117],
       [ 2.19168607, -2.92247203],
       [ 1.88922034, -2.9375098 ],
       [-0.88175568, -1.60171181],
       [-0.41897218, -0.21092846],
       [ 1.68026128,

In [78]:
LossERM(X, yNoisy, wvLearned[:,0], wvLearned[:,1], sigmaSquare, LambdaRegularization)

516271.88545690355

In [87]:
GradERM(X, yNoisy, wvLearned[:,0], wvLearned[:,1], sigmaSquare, DsigmaSquare, LambdaRegularization)

[-1.63294761e+07  1.32664264e+07  7.22918072e+07 -7.70764931e+06
 -2.07673412e+07 -8.98749479e+06  4.12247608e+07 -1.63113523e+07
 -5.20025326e+06  6.70049324e+06 -3.52358788e+07  4.74822686e+06
 -1.62306431e+07 -1.66361813e+06 -2.07667733e+07  1.36054063e+07
 -9.97118062e+06  3.03746902e+07 -2.82460860e+05 -1.61241008e+07
 -6.67277074e+06  7.43052824e+07  1.89078694e+07 -6.61882745e+07
  1.47459221e+07  4.64693309e+07  1.72004253e+07 -6.26065016e+07
  9.02284952e+07 -6.30680639e+07  5.41646687e+06  2.31203067e+06
  6.24450990e+06  2.26210625e+07  1.58254588e+07 -5.42759397e+06
 -3.21010693e+07  3.39055106e+07 -1.82615812e+07  3.59034126e+07
  1.61067783e+07  3.83690640e+07 -5.85479494e+06  2.10505960e+07
 -1.55648483e+06  5.14057003e+07  5.77090447e+07  1.12917806e+07
  4.40881129e+07 -6.03049227e+06  5.59134121e+06 -3.81164406e+07
 -6.78002037e+07 -5.19964815e+07 -6.35819887e+07  3.25029596e+06
  7.57909724e+07  5.68084214e+06  5.49762154e+07  1.13191274e+08
 -4.44215072e+07  3.53384

array([[ 6.81140922e+03, -1.63303777e+07],
       [-1.03565874e+04,  1.32677017e+07],
       [-4.25493638e+04,  7.22920206e+07],
       [ 8.47280758e+02, -7.70777771e+06],
       [ 3.95891785e+03, -2.07681034e+07],
       [ 1.79852371e+04, -8.98737560e+06],
       [-2.10681288e+04,  4.12250091e+07],
       [ 1.57874356e+04, -1.63094948e+07],
       [ 1.36948745e+04, -5.19871218e+06],
       [-5.46175443e+03,  6.69998650e+06],
       [ 3.51587932e+04, -3.52341546e+07],
       [ 2.64039816e+03,  4.74779194e+06],
       [-5.40989907e+02, -1.62302759e+07],
       [ 2.87768259e+03, -1.66418642e+06],
       [ 2.42397876e+04, -2.07672436e+07],
       [ 3.64212272e+03,  1.36052193e+07],
       [ 9.21448369e+03, -9.97017474e+06],
       [-2.62976863e+04,  3.03752590e+07],
       [ 1.37408562e+04, -2.82905012e+05],
       [ 1.48970502e+04, -1.61251224e+07],
       [ 1.59430506e+04, -6.67263221e+06],
       [-4.08575538e+04,  7.43060241e+07],
       [-2.11402579e+04,  1.89079107e+07],
       [ 3.